# Dockerfile

**Dockerfile 작성**

- 도커 엔진은 Dockerfile 을 읽어 들일 때 기본적으로 현재 디렉토리에 있는 Dockerfile 이라는 이름을 가진 파일을 선택한다.
- `FROM` : 생성할 이미지의 베이스가 될 이미지를 뜻한다.
- `MAINTAINER`: 이미지를 생성할 개발자의 정보
- `LABLE` : 이미지의 메타데이터를 추가. 메타데이터는 ‘키:값’ 형태로 저장
- `RUN`
    - 컨테이너 내부에서 명령어를 실행
    - 이미지를 빌드할 때 별도의 입력을 받아야 하는 RUN 이 있다면 build 명령어는 이를 오류로 간주하고 빌드를 종료한다.
- `ADD` : 파일을 이미지에 추가. 추가하는 파일은 Dockerfile 이 위치한 디렉토리인 컨텍스트에서 가져온다.
- `WORKDIR` : 명령어를 실행할 디렉터리를 나타낸다. 배시 셸에서 cd 명령어를 입력하는 것과 같은 기능
- `EXPOSE` : Dockerfile 의 빌드로 생성된 이미지에서 노출할 포트를 설정
- `CMD`
    - 컨테이너가 시작될 때마다 실행할 명령어를 설정. **Dockerfile 에서 한 번만 사용 할 수 있다.**
    - CMD 는 run 명령어의 이미지 이름 뒤에 입력하는 커맨드와 같은 역할을 하지만 docker run 명령어에서 커맨드 명령줄 인자를 입력하면 Dockerfile 에서 사용한 CMD 의 명령어는 run 의 커맨드로 엎어 쓰인다.



# Dockerfile 빌드

**이미지 생성**

- -t: 생성될 이미지의 이름을 설정

```bash
docker built -t mybuild:0.0 ./
```

**빌드 과정 살펴보기**

- 이미지 빌드를 시작하면 도커는 가장 먼저 `빌드 컨텍스트`를 읽어 들인다.
    - 빌드 컨텍스트는 이미지를 생성하는 데 필요한 각종 파일, 소스코드, 메타데이터 등을 담고 있는 디렉토리를 의미하며, Dockerfile 이 위치한 디렉토리가 빌드 컨텍스트가 된다.
- **컨텍스트는 build 명령어의 맨 마지막에 지정된 위치에 있는 파일을 전부 포함한다.**
    - Dockerfile 이 위치한 곳에는 이미지 빌드에 필요한 파일만 있는 것이 좋다.
    - 루드 디렉토리와 같은 곳에서 이미지를 빌드하지 않도록 주의해야 한다.
    - 컨텍스트는 단순 파일뿐 아니라 하위 디렉터리도 전부 포함되게 되므로 빌드에 불필요한 파일이 포함된다면 빌드 속도가 느려질뿐더러 호스트의 메모리를 지나치게 점유할수도 있다.
    - `.dockerignore`라는 파일을 작성하면 빌드 시 이 파일에 명시된 이름의 파일을 컨텍스트에서 제외한다.
        - 이 파일은 Dockerfile 이 위치한 경로와 같은 곳에 위치해야 한다.
        - 컨텍스트에서 제외할 파일의 경로는 Dockerfile 이 존재하는 경로를 기준으로 한다.
- Dockerfile 을 이용한 컨테이너 생성과 커밋
    - **Dockerfile 에서 명령어 한 줄이 실행될 때마다 이전 Step 에서 생성된 이미지에 의해 새로운 컨테이너가 생성되며, Dockerfile 에 적힌 명령어을 수행하고 다시 새로운 이미지 레이어로 저장된다.**
        
- 캐시를 이용한 이미지 빌드
    - 한 번 이미지 빌드를 마치고 난 뒤 다시 같은 빌드를 진행하면 이전의 이미지 빌드에서 사용했던 캐시를 사용한다.
    - 이미지 빌드 중 오류가 발생했을 때는 build 명령어가 중지되며, 이미지 레이어 생성을 위해 마지막으로 생성된 임시 컨테이너가 삭제되지 않은 채로 남게 된다.
    - `--no-cache`: 캐시를 사용하지 않을 때 사용하는 옵션
    - `--cache-from`: 캐시로 사용할 이미지를 직접 지정
        
        ```bash
        docker build --cache-from nginx -t my_extend_nginx:0.0 .
        ```
        

**멀티 스테이지를 이용한 Dockerfile 빌드하기**

- 17.05 버전 이상 도커 엔진이라면 이미지의 크기를 줄이기 위해 멀티 스테이지 빌드 방법을 사용할 수 있다.
- 멀티 스트에지 빌드는 하나의 Dockerfile 안에 여러 개의 FROM 이미지를 정의함으로써 빌드 완료 시 최종적으로 생성될 이미지의 크기를 줄이는 역할을 한다.
    - alpine 이나 busybox 와 같은 이미지는 우분투나 CentOS 에 비해 경량화된 리눅스 배포판 이미지이다.
    
    ```go
    package main
    import "fmt"
    
    func main() {
            fmt.Println("hello world")
    }
    ```
    
    ```bash
    FROM golang
    ADD main.go /root
    WORKDIR /root
    RUN go build -o /root/mainApp /root/main.go
    
    FROM alpine:latest
    WORKDIR /root
    COPY --from=0 /root/mainApp .
    CMD ["./mainApp"]
    ```
    
    ```bash
    docker build . -t go_helloworld:multi-stage
    ```
    
    - 멀티 스테이지 빌드는 반드시 필요한 실행 파일만 최종 이미지 결과물에 포함시킴으로써 이미지 크기를 줄일 때 유용하게 사용할 수 있다.
    - 각 이미지는 먼저 FROM 에서 명시된 순서대로 0,1… 의 순으로 차례대로 구분되어 사용된다.

**기타 Dockerfile 명령어**

**ENV, VOLUME, ARG, USER**

- `ENV`
    - Dockerfile 에서 사용될 환경 변수를 지정한다.
    - 설정한 환경변수는 ${ENV_NAME} 또는 $ENV_NAME 의 형태로 사용 가능
    - Dockerfile 에서 환경변수의 값을 사용할 때 배시 셸에서 사용하는 것처럼 값이 설정되지 않은 경우와 설정된 경우를 구분해 사용할 수 있다.
        - ${env_anme:-value}: env_name 이라는 환경변수의 값이 설정되지 않았으면 이 환경변수의 값을 value 로 사용한다.
        - ${env_name:+value}: env_name 의 값이 설정돼 있으면 value 를 값으로 사용하고, 값이 설정되지 않았다면 빈 문자열을 사용한다.
- `VOLUME`: 빌드된 이미지로 컨테이너를 생성했을 때 호스트와 공유할 컨테이너 내부의 디렉터리를 설정
- `ARG`
    - build 명령어를 실행할 때 추가로 입력을 받아 Dockerfile 내에서 사용될 변수의 값을 설정
    - `--build-arg` 옵션을 사용해 Dockerfile 의 ARG 에 값을 입력할 수 있다.
    
    ```bash
    docker buiild --build-arg my_arg=/home -t myarg:0.0 ./
    ```
    
    - Dockerfile 에서 ARG 로 설정한 변수를 ENV 에서 같은 이름으로 다시 정의하면 —build-arg 옵션에서 설정하는 값은 ENV 에 의해 덮어쓰여진다.
- `USER`
    - USER 로 컨테이너 내에서 사용될 사용자 계정의 이름이나 UID 를 설정하면 그 아래의 명령어는 해당 사용자 권한으로 실행된다.
    - 루트 권한이 필요하지 않다면 USER 를 사용하는 것을 권장한다.

**Onbuild, Stopsignal, Healthcheck, Shell**

- `ONBUILD`
    - 빌드된 이미지를 기반으로 하는 다른 이미지가 Dockerfile 로 생성될 때 실행할 명령어를 추가
    - ONBUILD 는 ONBUILD, FROM, MAINTAINER 를 제외한 RUN, ADD 등, **이미지가 빌드될 때 수행돼야 하는 각종 Dockerfile 의 명령어를 나중에 빌드될 이미지를 위해 미리 저장해 놓을 수 있다.**
    - ONBUILD 를 활용하는 좋은 방법 중 하나는 이미지가 빌드하거나 활용할 소스코드를 ONBUILD ADD 로 추가해서 좀 더 깔끔하게 Dockerfile 을 사용하는 것이다.
    - 예) 메이븐 도커 이미지
        
        ```bash
        FROM maven:3-jdk-8-alpine
        RUN mkdir -p /usr/src/app
        WORKDIR /usr/src/app
        ONBUILD ADD . /usr/src/app
        ONBUILD RUN mvn install
        ```
        
- `STOPSIGNAL`: 컨테이너가 정지될 때 사용될 시스템 콜의 종류를 지정
- `HEALTHCHECK`
    - 이미지로부터 생성된 컨테이너에서 동작하는 애플리케이션의 상태를 체크하도록 설정
    - HEALTHCHECK 에서 사용되는 명령어가 curl 이므로 컨테이너에 curl 을 먼저 설치해야 한다.
    
    ```bash
    FROM nginx
    RUN apt-get update -y && apt-get install curl -y
    HEALTHCHECK --interval=1m --timeout=3s --retries=3 CMD curl -f http://localhost || exit 1
    ```
    
- `SHELL`: 기본 셸을 설정
    
    ```bash
    FROM node
    RUN echo hello, node!
    SHELL ["/usr/local/bin/node"]
    RUN -v
    ```
    

**ADD, COPY**

- `COPY` 는 로컬 데릭터리에서 읽어 들인 컨텍스트로부터 이미지에 파일을 복사하는 역할을 한다.
- COPY 는 로컬의 파일만 이미지에 추가할 수 있지만 ADD 는 외부 URL 및 tar 파일에서도 파일을 추가할 수 있다는 점에서 다르다.
- **ADD 를 사용하는 것은 그다지 권장하지 않는다. ADD 로 URL 이나 tar 파일을 추가할 경우 이미지에 정확히 어떤 파일이 추가될지 알 수 없기 때문이다.**

**ENTRYPOINT, CMD**

- ENTRYPOINT 와 CMD 의 차이점
    - entrypoint 는 커맨드와 동일하게 컨테이너가 시작될 때 수행할 명령을 지정한다는 점에서 같다. 그러나 entrypoint 는 커맨드를 인자로 받아 사용할 수 있는 스크립트의 역할을 할 수 있다는 점에서 다르다.
    - **entrypoint 가 설정되지 않았다면 cmd 에 설정된 명령어를 그대로 실행하지만 entrypoint 가 설정 됐다면 cmd 는 단지 entrypoint 에 대한 인자의 기능을 한다.**
    - 반드시 둘중 하나를 설정해야 한다.
    - 일반적으로는 스크립트 파일을 entrypoint 의 인자로 사용해 컨테이너가 시작될 때마다 해당 스크립트 파일을 실행하도록 설정한다.
        - 단 실행할 스크립트 파일은 컨테이너 내부에 존재해야 한다.


# 도커 데몬

# 도커의 구조

- 리눅스의 `which` 명령어는 파일이 위치한 경로를 출력
- `ps aux` 명령어는 실행 중인 프로세스의 목록을 출력
- 도커의 구조는 크게 두 가지로 나뉜다. 하나는 클라이언트로서의 도커이고, 다른 하나는 서버로서의 도커이다.
- 실제로 컨테이너를 생성하고 실행하며 이미지를 관리하는 주체는 도커 서버이고, 이는 `dockerd` 프로세스로서 동작한다.
- 도커 엔진은 외부에서 API 입력을 받아 도커 엔진의 기능을 수행하는데, 도커 프로세스가 실행되어 서버로서 입력을 받을 준비가 된 상태를 도커 데몬이라고 이야기한다.

**도커 데몬 실행**

- 실제 운영 환경에서는 도커 데몬을 직접 실행하기보다는 service, systemctl 명령어를 통해 리눅스 서비스로서 관리하는 것이 좋다.

**도커 데몬 설정**

- 우분투 기준으로 `/lib/systemd/system/docker.service` 파일을 열어 보면 `/etc/default/docker` 여기에 설정해도 딱히 읽는거 같지 않다;;;
- 다른 방안으로 설정파일 대신 docker service 를 내리고 dockerd 로 실행하고 다음 실습을 이어서 진행한다.
    
    ```bash
    sudo dockerd -H tcp://0.0.0.0:2375
    
    # 위의 명령어를 실행한 터미널은 포그라운드 상태로 실행중이므로 다른 터미널 열어서 테스트 
    curl localhost:2375/version
    ```
    

**도커 데몬 제어: -H**

- -H 옵션은 도커 데몬의 API 를 사용할 수 있는 방법을 추가한다.
- 아무런 옵션을 설정하지 않고 도커 데몬을 실행하면 도커 클라이언트인 /usr/bin/docker 를 위한 유닉스 소켓인 /var/run/docker.sock 를 사용한다.
- -H 에 IP 주소와 포트 번호를 입력하면 원격 API 인 Docker Remote API 로 도커를 제어할 수 있다.
- Remote API 만을 위한 바인딩 주소를 입력했다면 유닉스 소켓은 비활성화되므로 도커 클라이언트를 사용할 수 없게 되며, docker 로 시작하는 명령어를 사용할 수 없다.
- 따라서 일반적으로 도커 클라이언트를 위한 유닉스 소켓과 Remote API 를 위한 바인딩 주소를 동시에 설정한다.
    
    ```bash
    dockerd -H unix:///var/rundocker.sock -H tcp://0.0.0.0:2375
    ```
    
- **도커 클라이언트는 셸의 `DOCKER_HOST` 변수가 설정돼 있다면 해당 도커 데몬에 API 요청을 전달한다.**

**도커 데몬에 보안 적용: —tlsverify**

- 보안에 적용할 때 사용될 파일은 총 5개로서, ca.pem, server-cert.pem, server-key.pem, cert.pem, key.pem 이다.
- 클라이언트 측에서 도커 데몬에 접근하라면 ca.pem, cert.pem, key.pem 파일이 필요하다.
- 우분투 openssl 설치 오류 관련?
    - ldconfig /usr/local/lib64/


1. 서버 측 파일 생성

```bash
mkdir keys && cd keys

# 인증서에 사용될 키를 생성
openssl genrsa -aes256 -out ca-key.pem 4096

# 공용키 생성(위에서 입력한 비밀번호를 입력한다.)
openssl req -new -x509 -days 10000 -key ca-key.pem -sha256 -out ca.pem

# 서버 측에서 사용될 키를 생성
openssl genrsa -out server-key.pem 4096

# 외부에서 접근 가능한 IP 설정은 ngrok 사용 
./ngrok tcp 2375

# 서버 측에서 사용될 인증서를 위한 인증 요청서 파일을 생성 
# $HOST 부분에는 사용 중인 도커 호스트의 IP 주소 또는 도메인 이름을 입력하며, 이는 외부에서 접근 가능한 IP 주소 또는 도메인 이름이어야 한다.
# 여기서는 ngrock 으로 포워딩한 도메인 주소를 적어준다.
openssl req -subj "/CN=0.tcp.jp.ngrok.io:16280" -sha256 -new -key server-key.pem -out server.csr

# 위에 입력해서 RANDFILE 관련 오류가 난다면 /etc/ssl/openssl.cnf 에서 RANDFILE 경로 부분을 주석해주고 다시 실행 
# 접속에 사용될 IP 주소를 extfile.cnf 파일로 저장
echo subjectAltName = IP:0.tcp.jp.ngrok.io:16280,IP:127.0.0.1 > extfile.cnf

# 서버 인증서 생성
openssl x509 -req -days 365 -sha256 -in server.csr -CA ca.pem -CAkey ca-key.pem -CAcreateserial -out server-cert.pem -extfile extfile.cnf
```

1. 클라이언트 측에서 사용할 파일 생성 

```bash
# 클라이언트 측의 키 파일과 인증 요청 파일 생성
openssl genrsa -out key.pem 4096
openssl req -subj '/CN=client' -new -key key.pem -out client.csr

# 클라이언트 측의 인증서를 생성
openssl x509 -req -days 30000 -sha256 -in client.csr -CA ca.pem -CAkey ca-key.pem -CAcreateserial -out cert.pem

# 생성된 파일의 쓰기 권한을 삭제해 읽기 전용 파일로 만든다
chmod -v 0400 ca-key.pem key.pem server-key.pem ca.pem server-cert.pem cert.pem

# 도커 데몬의 설정파일이 존재하는 ~/.docker 로 파일을 옮긴다.
cp {ca,server-cert,server-key,cert,key}.pem ~/.docker

# 암호화가 적용된 도커 데몬 실행 
**# 도커의 Remote API 를 사용하는 포트는 보안이 X 라면 2375, 보안이 O 라면 2376을 사용하는게 관례다.**
sudo dockerd --tlsverify --tlscacert=/home/shpark/.docker/ca.pem --tlscert=/home/shpark/.docker/server-cert.pem --tlskey=/home/shpark/.docker/server-key.pem -H=0.0.0.0:2375 -H unix:///var/run/docker.sock

# TLS 연결 설정을 하지 않았다는 에러 출력
docker -H tcp://0.tcp.jp.ngrok.io:13577 version

# 보안이 적용된 도커 데몬을 사용하려면 ca.pem, key.pem, cert.pem 파일이 필요하다.
docker -H tcp://0.tcp.jp.ngrok.io:13577 --tlscacert=/home/shpark/.docker/ca.pem --tlscert=/home/shpark/.docker/cert.pem --tlskey=/home/shpark/.docker/key.pem --tlsverify version
```

**도커 스토리지 드라이버 변경: —storage-driver**

- 도커를 사용하는 환경에 따라 스토리지 드라이버는 자동으로 정해지지만 도커 데몬 실행 옵션에서 스토리지 드라이버를 변경할 수도 있다.

#  도커 데몬 모니터링

**도커 데몬 디버그 모드**

```kotlin
dockerd -D
```

**events, stats, system df 명령어**

- events
    - 도커 데몬에 어떤 일이 일어나고 있는지를 실시간 스트림 로그로 보여준다.
    - attach, commit, copy, create 등의 컨테이너 관련 명령어, delete, import, load, pull, push 등의 이미지 관련 명령어, 볼륨, 네트워크, 플러그인 등에 관한 명령어의 수행 결과가 출력된다.
    
    ```kotlin
    docker events
    docker system events
    docker events --filter 'type=image'
    ```
    
- stats
    - 실행 중인 모든 컨테이너의 자원 사용량을 스트림으로 출력한다.
    - 실행 중인 모든 컨테이너의 CPU, 메모리 제한 및 사용량, 네트워크 입출력(I/O), 블록 입출력(하드웨어 입출력) 정보를 출력한다.
    
    ```kotlin
    docker stats
    ```
    
- system df
    - 도커에서 사용하고 있는 이미지, 컨테이너, 로컬 볼륨의 총 개수 및 사용 중인 개수, 크기, 삭제함으로써 확보 가능한 공간을 출력한다.
    
    ```kotlin
    docker system df
    ```
    

**CAdvisor**

- 구글이 만든 컨테이너 모니터 도구로, 컨테이너로서 간단히 설치할 수 있고 컨테이너별 실시간 자원 사용량 및 도커 모니터링 정보 등을 시각화해서 보여준다.

```kotlin
docker run --volume=/:/rootfs:ro --volume=/var/run:/var/run:ro --volume=/sys:/sys:ro --volume=/var/lib/docker/:/var/lib/docker:ro --volume=/dev/disk/:/dev/disk:ro --publish=18080:8080 --detach=true --name=cadvisor google/cadvisor:latest
```

- CAdvisor 의 대시보드는 60초간의 모니터링 정보만 보여주지만 InfluxDB 나 Prometheus 등과 같이 사용하면 장기간의 모니터링 정보를 수집하고 분석할 수 있다.
- `/sys` 에는 도커 컨테이너를 위한 cgroup 정보가 저장돼 있으며 `/var/lib/docker` 에는 도커의 컨테이너, 이미지 등이 파일로 존재한다.